# 4. Clean the data
This checks the links between Gutenberg text and Goodreads data, and drops books that don't match. This trimmed down data becomes the main book list for the project.

In [ ]:
import re
import pandas as pd

from pathlib import Path

data_dir = Path.cwd().parent / "data"
fiction = pd.read_csv(data_dir / "book_data_full.csv").sort_values("Catalogue number")
print(fiction.shape)

# REMOVALS
# Remove anything with no reviews on Goodreads
fiction = fiction[fiction['Review count'] >= 1]
print(fiction.shape)

# Remove anything with no author on Gutenberg
fiction = fiction[fiction['Gutenberg author'].notnull()]
print(fiction.shape)

# Find duplicates in Gutenberg and keep higher catalogue number
# Higher catalogue number usually means a corrected text
fiction.drop_duplicates(
    subset=["Gutenberg title", "Gutenberg author"], 
    keep="last",
    inplace=True)
print(fiction.shape)

# Find duplicates in Goodreads and keep higher Gutenberg catalogue number
fiction.drop_duplicates(
    subset=["Goodreads title", "Goodreads author"], 
    keep="last",
    inplace=True)
print(fiction.shape)

# EDITS 
# Remove series brackets from Goodreads titles
fiction["Goodreads title"] = fiction["Goodreads title"].str.replace("\(.*\)", "")

# Remove line breaks from Gutenberg titles
fiction["Gutenberg title"] = fiction["Gutenberg title"].str.replace("\\r\\n", " ")
fiction["Gutenberg title"] = fiction["Gutenberg title"].str.replace("\n", " ")

In [ ]:
# Try to find where Goodreads title and author don't match Gutenberg
# Start by counting the number of matching words in the author and title fields
punctuation = re.compile("[^\w\s]")
def word_matches(x1, x2):
    x1_edit = re.sub(punctuation, " ", x1.lower())
    x1_split = set([w for w in x1_edit.split(" ") if len(w) > 1])
    x2_edit = re.sub(punctuation, " ", x2.lower())
    x2_split = set([w for w in x2_edit.split(" ") if len(w) > 1])
    return len(x1_split & x2_split)

fiction['Name matches'] = fiction.apply(
    lambda row: word_matches(
        row["Goodreads author"], 
        row["Gutenberg author"]), axis=1)

fiction['Title matches'] = fiction.apply(
    lambda row: word_matches(
        row["Goodreads title"], 
        row["Gutenberg title"]), axis=1)

In [ ]:
# Require at least one matching word in title and one in author
# Loose requirement as often the same book is listed with variant titles or author spelling
# Reasonable chance of losing some authors, for example based on accents in names
# But this is fine - there are thousands of books here and losing some is no problem
fiction = fiction[(fiction["Name matches"] > 0) & (fiction["Title matches"] > 0)]
fiction.shape

In [ ]:
fiction.to_csv(data_dir / "book_data_cut.csv", index=False)